<a href="https://colab.research.google.com/github/falco-tigris/TRANSFORMERS_EXAMPLES/blob/main/TRANSFORMRS_FROM_SCRATCH_HUGGINGFACE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.0 MB/s 
     |████████████████████████████████| 115 kB 71.2 MB/s 
     |████████████████████████████████| 212 kB 65.8 MB/s 
     |████████████████████████████████| 120 kB 70.4 MB/s 
     |████████████████████████████████| 127 kB 70.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import datasets

In [5]:
all_ds = datasets.list_datasets()
len(all_ds)

9106

In [6]:
all_ds[:5]

['acronym_identification',
 'ade_corpus_v2',
 'adversarial_qa',
 'aeslc',
 'afrikaans_ner_corpus']

In [8]:
dataset = datasets.load_dataset('oscar','unshuffled_deduplicated_la')

Generating train split:   0%|          | 0/18808 [00:00<?, ? examples/s]

Dataset oscar downloaded and prepared to /root/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_la/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 18808
    })
})

In [10]:
dataset['train'][0]

{'id': 0,
 'text': 'Hæ sunt generationes Noë: Noë vir justus atque perfectus fuit in generationibus suis; cum Deo ambulavit.\nEcce ego adducam aquas diluvii super terram, ut interficiam omnem carnem, in qua spiritus vitæ est subter cælum: universa quæ in terra sunt, consumentur.\nTolles igitur tecum ex omnibus escis, quæ mandi possunt, et comportabis apud te: et erunt tam tibi, quam illis in cibum.'}

In [11]:
#Save data in files instead of temporary storage like a variable for tokenization
from tqdm.auto import tqdm
text_data = []
file_count=0
for sample in tqdm(dataset['train']):
  sample = sample['text'].replace('\n',' ')
  text_data.append(sample)
  if len(text_data)==10_000:
    with open(f'la_{file_count}.txt','w',encoding='utf-8') as fp:
      fp.write('\n'.join(text_data))
    text_data = []
    file_count+=1
with open(f'la_{file_count}.txt','w',encoding='utf-8') as fp:
  fp.write('\n'.join(text_data))
   
      
  

  0%|          | 0/18808 [00:00<?, ?it/s]

# **TOKENIZATION**

In [13]:
from pathlib import Path

In [14]:
paths = [str(x) for x in Path('./').glob('*.txt')]
paths[:2]

['la_0.txt', 'la_1.txt']

In [15]:
#BPETokenizer - no need for UNK token, breaking down to bytes

In [16]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 4.1 MB/s 


In [17]:
from tokenizers import ByteLevelBPETokenizer

In [18]:
tokenizer = ByteLevelBPETokenizer()

In [20]:
tokenizer.train(files=paths,vocab_size=30_522,min_frequency=2,special_tokens=['<s>','<pad>','</s>','<unk>','<mask>'])

In [22]:
import os
os.mkdir('latinberto')

In [23]:
tokenizer.save_model('/content/latinberto')

['/content/latinberto/vocab.json', '/content/latinberto/merges.txt']